<a href="https://colab.research.google.com/github/Miku0204/Michigan-Computer-Vision/blob/main/KNN%20Classifier/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [17]:
%cd ..


/content


In [18]:
%cd Michigan-CV

/content/Michigan-CV


In [20]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	KNN Classifier/

nothing added to commit but untracked files present (use "git add" to track)


In [21]:
!git add .

In [22]:
!git commit -m "Add Data files"

[master (root-commit) 080c5f7] Add Data files
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 KNN Classifier/utils/data.ipynb


In [25]:
%cd ../Michigan-Computer-Vision/

/content/Michigan-Computer-Vision


In [26]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	KNN Classifier/

nothing added to commit but untracked files present (use "git add" to track)


In [27]:
!git add .
!git commit -m "Add data file"

[main 128b914] Add data file
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 KNN Classifier/utils/data.ipynb


In [28]:
!git push

Counting objects: 5, done.
Writing objects: 100% (5/5), 335 bytes | 335.00 KiB/s, done.
Total 5 (delta 0), reused 0 (delta 0)
To https://github.com/Miku0204/Michigan-Computer-Vision
   e48b4b3..128b914  main -> main
